### Установка и импорт всех необходимых зависимостей

In [ ]:
!pip install -q razdel
!pip install -q pymorphy2
!pip install -q git+https://github.com/ahmados/rusynonyms.git
!pip install -q natasha

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re
import pymorphy2
from razdel import tokenize
from razdel import sentenize
import string
from natasha import (
    MorphVocab,
    NewsMorphTagger,
    NewsEmbedding,
    Segmenter,
    NewsSyntaxParser,
    Doc
)

import torch
import tensorflow_hub as hub
from torch import nn
from torch.utils.data import Dataset, DataLoader
import transformers
import numpy as np

from tqdm import tqdm
import os
import sys
from typing import *

from lime.lime_text import LimeTextExplainer
import shap

nltk.download('stopwords')
nltk.download('punkt')
rus_stopwords = stopwords.words('russian')
punctuation = list(string.punctuation)

### Работа с данными (kaggle)

In [ ]:
datasets_folder = '/kaggle/input/sw-datasets/Russian-Sentiment-Analysis-Evaluation-Datasets'
datasets = ['SentiRuEval-2015-telecoms', 'SentiRuEval-2015-banks', 'SentiRuEval-2016-banks', 'SentiRuEval-2016-telecoms']
samples = ['test.xml', 'train.xml', 'test_etalon.xml']

In [ ]:
def extract_data(path: str) -> pd.DataFrame:
    """
    функция для извлечения данных из xml
    """
    tree = ET.parse(path)
    root = tree.getroot()
    DataFrame = dict()
    database = root.findall('database')[0]
    DataFrame_columns = list()

    for idx, table in enumerate(database.findall('table')):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']] = list()
            DataFrame_columns.append(column.attrib['name'])
        if idx == 0:
            break

    for table in database.findall('table'):
        for column in table.findall('column'):
            DataFrame[column.attrib['name']].append(column.text)

    data = pd.DataFrame(DataFrame, columns=DataFrame_columns)
    return data

# инициализация всех путей (kaggle)
banks_dataset = datasets[2]
path2samples = os.path.join(datasets_folder, banks_dataset)
banks = ['sberbank', 'vtb', 'gazprom', 'alfabank', 'bankmoskvy', 'raiffeisen', 'uralsib', 'rshb']

path2test = os.path.join(path2samples, samples[2])
data_test = extract_data(path2test)

path2train = os.path.join(path2samples, samples[1])
data_train = extract_data(path2train)

In [ ]:
def extract_text_features(data: pd.DataFrame) -> pd.DataFrame:
    """
    функция для первичной обработки текста от лишних символов
    """
    extracted_data = dict()
    extracted_data['text'] = list()
    extracted_data['0class'] = list()
    extracted_data['1class'] = list()

    for idx in range(len(data)):
        row = data.iloc[idx, :]
        banks_review = row[banks]
        unique_labels = set(banks_review)
        unique_labels.remove('NULL')

        # убираем все ненужные знаки
        filtered_text = re.sub('http[A-z|:|.|/|0-9]*', '', row['text']).strip()
        filtered_text = re.sub('@\S*', '', filtered_text).strip()
        filtered_text = re.sub('#', '', filtered_text).strip()
        new_text = filtered_text

        # сохраняем только уникальные токены (без придатка xml NULL)
        unique_labels = list(unique_labels)
        while len(unique_labels) < 2:
            unique_labels.append(unique_labels[-1])
        extracted_data['text'].append(new_text)
        for idx, label in enumerate(unique_labels):
            text_label = int(label) + 1
            extracted_data[f'{idx}' + 'class'].append(text_label)

    extracted_data = pd.DataFrame(extracted_data)
    
    # возвращаем dataframe
    return extracted_data

extracted_test = extract_text_features(data_test)
extracted_train = extract_text_features(data_train)

In [ ]:
# пример твита из датасета
extracted_test.iloc[3308].text

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# анализ распределения таргетов на твитах
fig, axes = plt.subplots(1, 2, figsize=(8, 5))
plt.subplots_adjust(hspace=0.15, wspace=0.3)

graph1 = sns.countplot(data=extracted_train, x='0class', ax=axes[0])
graph1.set(xlabel='class_num', ylabel='amount of class', title='Amount of classes according 1 label')
graph1.grid(True)

graph2 = sns.countplot(data=extracted_train, x='1class', ax=axes[1])
graph2.set(xlabel='class_num', ylabel='amount of class', title='Amount of classes according 2 label')
graph2.grid(True)

None

### Инициализируем модель (fine-tune) для решения нашей задачи классификации

In [ ]:
learning_rate = 1e-05


class BERTmy(torch.nn.Module):
    def __init__(self, n_classes: int) -> None:
        super(BERTmy, self).__init__()
        self.rubert = transformers.AutoModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence"
        )
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", 
            do_lower_case=True,
            add_additional_tokens=True
        )
        
        hidden_size_output = self.rubert.config.hidden_size
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_size_output, hidden_size_output, bias=True),
            torch.nn.Dropout(0.05),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size_output, n_classes),
        )

    def forward(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor, 
        token_type_ids: torch.Tensor, output_attentions: bool=False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        rubert_output = self.rubert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
            output_attentions=output_attentions
        )
        if not output_attentions:
            pooled = rubert_output['pooler_output']
        else:
            pooled, attentions = rubert_output['pooler_output'], rubert_output['attentions']

        output = self.classifier(pooled)

        if not output_attentions:
            return output
        else:
            return output, attentions
    
    def configure_optimizer(
        self, use_scheduler: bool=False
    ) -> torch.optim:
        # freeze part of params
        encoder_size = 0
        for param in self.rubert._modules['encoder'].parameters():
            encoder_size += 1
        encoder_size_half = encoder_size // 2
        for idx, param in enumerate(self.rubert._modules['encoder'].parameters()):
            param.requires_grad = False
            if idx >= encoder_size_half:
                break
        
        # Adam
        optimizer = torch.optim.Adam(
            params=[
                {'params':self.rubert._modules['embeddings'].parameters(), 'lr':4e-6},
                {'params':self.rubert._modules['encoder'].parameters(), 'lr':4e-6},
                {'params':self.rubert._modules['pooler'].parameters(), 'lr':4e-6},
                {'params':self.classifier.parameters(), 'lr':9e-5}
            ],
            lr=learning_rate
        )
        if use_scheduler:
            # scheduler
            scheduler = torch.optim.lr_scheduler.ExponentialLR(
                optimizer, gamma=0.96
            )
        
            return optimizer, scheduler
        
        else:
            return optimizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_cls = len(pd.unique(extracted_train['0class']))
bert = BERTmy(num_cls)
if torch.cuda.is_available():
    bert = bert.cuda()
optimizer, scheduler = bert.configure_optimizer(use_scheduler=True)

### Инициализируем class для нашего датасета

In [ ]:
train_batch_size = 32
val_batch_size = 16

class SentimentData(Dataset):
    # инициализация датасета
    def __init__(
        self, dataframe: pd.DataFrame, mode: str, 
        col_name: str, split_param: float=0.9
    ) -> None:
        self.mode = mode # train/test
        self.data = dataframe # data
        self.col_name = col_name # column for analyzing
        
        data_size = self.data.shape[0]
        if self.mode in ['val', 'train']:
            if self.mode == 'train':
                self.data = self.data.iloc[:int(data_size * split_param)]
            else:
                self.data = self.data.iloc[int(data_size * split_param):]
        
        assert self.mode in ['val', 'train', 'test']

    # для получения размера датасета
    def __len__(self) -> int:
        return self.data.shape[0]

    # для получения элемента по индексу
    def __getitem__(
        self, index: int
    ) -> Dict[str, Union[str, torch.Tensor]]:
        text = self.data.iloc[index][self.col_name]
        target1 = self.data.iloc[index]['0class']
        target2 = self.data.iloc[index]['1class']

        return {
            'text': text,
            'target1': torch.tensor(target1, dtype=torch.long),
            'target2': torch.tensor(target2, dtype=torch.long)
        }

### Инициализируем наши DataLoaders

In [ ]:
train = SentimentData(
    dataframe=extracted_train,
    split_param=1.0,
    mode='train',
    col_name='text'
)

val = SentimentData(
    dataframe=extracted_train,
    mode='val',
    col_name='text'
)

test = SentimentData(
    dataframe=extracted_test,
    mode='test',
    col_name='text'
)

train_loader = DataLoader(train, batch_size=train_batch_size, shuffle=True)
# val_loader = DataLoader(val, batch_size=val_batch_size, shuffle=False)
loaders = {
    'train': train_loader,
    # 'val': val_loader
}

### Дообучение модели

In [ ]:
rubert_tokenizer = bert.tokenizer


def train_model(
    epochs: int, model: torch.nn.Module, loaders: List[DataLoader], 
    optimizer: torch.optim, scheduler: torch.optim.lr_scheduler
) -> torch.nn.Module:
    # cross entropy loss
    loss_function1 = torch.nn.CrossEntropyLoss()
    loss_function2 = torch.nn.CrossEntropyLoss()
    
    # извлечение DataLoaders
    if len(loaders) > 1:
        train_loader = loaders['train']
        val_loader = loaders['val']
        steps_per_epoch = [('train', train_loader), ('val', val_loader)]
    else:
        train_loader = loaders['train']
        steps_per_epoch = [('train', train_loader)]

    # обучение по эпохам
    for epoch in range(epochs):
        for mode, loader in steps_per_epoch:
            # сохранение статистик
            train_loss = 0
            n_correct = 0
            processed_data = 0
            
            # train/val 
            if mode == 'train':
                model.train()
                requires_grad_mode = True
            else:
                model.eval()
                requires_grad_mode = False
            
            # проход по батчам
            for data in tqdm(loader):
                # обнуляем градиенты
                optimizer.zero_grad()

                # извлечение входных данных для модели
                inputs = rubert_tokenizer(
                    data['text'], padding=True, truncation=True, 
                    add_special_tokens=True, return_tensors='pt'
                )
                ids = inputs['input_ids'].to(device)
                mask = inputs['attention_mask'].to(device)
                token_type_ids = inputs["token_type_ids"].to(device)
                target1 = data['target1'].to(device)
                target2 = data['target2'].to(device)
                
                # устанавливаем необходимость вычислять/не_вычислять градиенты
                with torch.set_grad_enabled(requires_grad_mode):
                    outputs = model(ids, mask, token_type_ids)
                    preds = torch.argmax(outputs.data, dim=1)

                    # настраиваем модели на конкретный target
                    if all(target1 == target2):
                        loss1 = loss_function1(outputs, target1)
                        train_loss += loss1.item() * outputs.size(0)
                        n_correct += torch.sum(preds == target1)
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss1.backward()
                            optimizer.step()
                    # если у твита более чем 1 метка, то настраиваем на обе
                    else:
                        loss1 = loss_function1(outputs, target1) * 0.5
                        loss2 = loss_function2(outputs, target2) * 0.5
                        loss_all = loss1 + loss2
                        train_loss += loss_all.item() * outputs.size(0)

                        mask_singular = target1 == target2
                        mask_multiple = target1 != target2
                        singular = preds[mask_singular]
                        n_correct += torch.sum(singular == target1[mask_singular])
                        multiple = preds[mask_multiple]
                        n_correct += torch.sum((multiple == target1[mask_multiple]) & (multiple == target2[mask_multiple]))
                        if mode == 'train':
                            # вычисляем градиенты и обновляем веса
                            loss_all.backward()
                            optimizer.step()     
                    processed_data += outputs.size(0)

            # вычисляем ошибку и точность прогноза на эпохе
            loader_loss = train_loss / processed_data
            loader_acc = n_correct.cpu().numpy() / processed_data
            print(f'{epoch + 1} epoch with {mode} mode has: {loader_loss} loss, {loader_acc} acc')
        
        # делаем шаг для sheduler оптимайзера
        scheduler.step()

    return model

In [ ]:
epochs = 12
bert = train_model(epochs, bert, loaders, optimizer, scheduler)

In [ ]:
mode_process = input('Load weights? (y/n)')
if mode_process == 'n':
    torch.save(bert.state_dict(), 'bert_weights_pooled.pth')
elif mode_process == 'y':
    bert.load_state_dict(torch.load('/kaggle/input/bert-weights-better/bert_weights_pooled.pth'))
else:
    assert mode_process in ['n', 'y']
bert.eval()
None

### Вычисление итоговых показателей

In [ ]:
def calculate_accuracy(
    model: torch.nn.Module, SentimentData:Dataset
) -> float:
    model.eval()
    loader = DataLoader(SentimentData, batch_size=10, shuffle=False)
    n_correct = 0
    processed_data = 0
    
    for data in tqdm(loader):
        inputs = model.tokenizer(
            data['text'], padding=True, 
            add_special_tokens=True, return_tensors='pt'
        )
        ids = inputs['input_ids'].to(device)
        mask = inputs['attention_mask'].to(device)
        token_type_ids = inputs["token_type_ids"].to(device)
        target1 = data['target1'].to(device)
        target2 = data['target2'].to(device)
        
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            preds = torch.argmax(outputs.data, dim=1)
            mask_singular = target1 == target2
            mask_multiple = target1 != target2
            singular = preds[mask_singular]
            n_correct += torch.sum(singular == target1[mask_singular])
            multiple = preds[mask_multiple]
            if len(multiple) > 0:
                n_correct += torch.sum((multiple == target1[mask_multiple]) & (multiple == target2[mask_multiple]))
            processed_data += outputs.size(0)
        
    loader_acc = n_correct.cpu().numpy() / processed_data
    
    return loader_acc

def calculate_f1_class(
    model: torch.nn.Module, SentimentData: Dataset, class_num: int
) -> float:
    model.eval()
    loader = DataLoader(SentimentData, batch_size=10, shuffle=False)
    true_positive = 0
    false_positive, false_negative = 0, 0
    
    for data in tqdm(loader):
        inputs = model.tokenizer(
            data['text'], padding=True, 
            add_special_tokens=True, return_tensors='pt'
        )
        ids = inputs['input_ids'].to(device)
        mask = inputs['attention_mask'].to(device)
        token_type_ids = inputs["token_type_ids"].to(device)
        target1 = data['target1'].to(device)
        target2 = data['target2'].to(device)
        
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            
            preds = torch.argmax(outputs.data, dim=1)
            preds = preds.cpu().numpy()
            target1 = target1.cpu().numpy()
            
            mask_positive = target1 == class_num
            mask_negative = target1 != class_num
            
            true_positive += np.sum(preds[mask_positive] == class_num)
            false_positive += np.sum(preds[mask_negative] == class_num)
            false_negative += np.sum(preds[mask_positive] != class_num)
        
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    loader_f1 = 2 * precision * recall / (precision + recall)
    
    return loader_f1

In [ ]:
test_acc = calculate_accuracy(bert, test)
class_neg_f1 = calculate_f1_class(bert, test, 0)
class_neu_f1 = calculate_f1_class(bert, test, 1)
class_pos_f1 = calculate_f1_class(bert, test, 2)

In [ ]:
# общая accuracy и f1 по классам
test_acc, class_neg_f1, class_neu_f1, class_pos_f1

### Backdoor attacks on neural network(adversial examples)

#### USE metric for similarity between original sentence and spoiled sentence

In [ ]:
def use_score(original, adversial, use_bert_encoder=False, model=None):
    from scipy.spatial.distance import cosine
    # Load pre-trained universal sentence encoder model
    if not use_bert_encoder:
        # using DAN from tensorflow
        use_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

        sentences_orig = list()
        sentences_adv = list()
        for pair in zip(original, adversial):
            orig, adv = pair
            sentences_orig.append(orig)
            sentences_adv.append(adv)

        # get embs of texts
        sentences_orig_emb = use_encoder(sentences_orig)
        sentences_adv_emb = use_encoder(sentences_adv)

        # calculate use_score with DAN
        use_scores = list()
        for pair in zip(sentences_orig_emb, sentences_adv_emb):
            orig_emb, adv_emb = pair[0], pair[1]
            use_score_one = 1 - cosine(orig_emb, adv_emb)
            use_scores.append(use_score_one)
    else:
        # using BERT itself
        def get_inputs(text): # get inputs for model
            inputs = model.tokenizer(
                text, padding=True, 
                add_special_tokens=True, 
                return_tensors='pt'
            )
            ids = inputs['input_ids'].type(torch.long).to(device)
            mask = inputs['attention_mask'].type(torch.long).to(device)
            token_type_ids = inputs["token_type_ids"].type(torch.long).to(device)
            
            return ids, mask, token_type_ids

        # calculate use_score with BERT
        use_scores = list()
        for pair in zip(original, adversial):
            orig, adv = pair[0], pair[1]
            orig_inputs = get_inputs(orig)
            adv_inputs = get_inputs(adv)
            orig_outputs = model.rubert(*orig_inputs)
            adv_outputs = model.rubert(*adv_inputs)
            orig_pooled, adv_pooled = orig_outputs[1], adv_outputs[1]
            orig_pooled = orig_pooled.cpu().detach().numpy()
            adv_pooled = adv_pooled.cpu().detach().numpy()
            use_score_one = 1 - cosine(orig_pooled, adv_pooled)
            use_scores.append(use_score_one)
    
    return use_scores, np.mean(use_scores)

### OrderBkd

In [ ]:
def order_bkd_extract(dataframe, col_name):
    order_spoiled_text = list()
    # союзы, наречия и предлоги
    special_units = ['CCONJ', 'SCONJ', 'PRON', 'ADV']
    # natasha's embs
    emb = NewsEmbedding()
    # морфологический анализатор
    morph_vocab = MorphVocab()
    # natasha's morph tagger
    morph_tagger = NewsMorphTagger(emb)
    # natasha's syntax parser
    syntax_parser = NewsSyntaxParser(emb)
    # natasha's segmenter
    segmenter = Segmenter()
    main_id = '1_0'
    source = dataframe[col_name]
    
    for idx in tqdm(range(len(source))):
        text = source.iloc[idx]
        # инициализация natasha's Doc
        doc_text = Doc(text)
        doc_text.segment(segmenter)
        doc_text.tag_morph(morph_tagger)
        doc_text.parse_syntax(syntax_parser)
        non_morphological_token = None
        morphological_dependece = None
        for token in doc_text.tokens:
            # лемматизированное слово
            token.lemmatize(morph_vocab)
            pos_tag = token.pos
            # если специальное слово и мы его еще не нашли
            if pos_tag in special_units and non_morphological_token is None:
                try:
                    non_morphological_token = (token.start, token.stop, token.text)
                except:
                    non_morphological_token = (0, token.stop, token.text)
            # если еще не нашли слово с морфологической зависимостью
            elif morphological_dependece is None:
                lower_token = token.text.lower()
                token_head_id = token.head_id
                token_id = token.id
                token_lemma = token.lemma
                # если у слова есть морфологическая зависимость
                if lower_token != token_lemma and token_id != token_head_id and token_head_id != main_id:
                    try:
                        morphological_dependece = (token.start, token.stop, token.text)
                    except:
                        morphological_dependece = (0, token.stop, token.text)
        # если нашли 2 слова на замену друг другу
        if not morphological_dependece is None and not non_morphological_token is None:
            text_symbols = list(text)
            start_dep, stop_dep = morphological_dependece[0], morphological_dependece[1]
            token_dep = morphological_dependece[2]
            start_non, stop_non = non_morphological_token[0], non_morphological_token[1]
            token_non = non_morphological_token[2]
            
            # меняем их местами
            start_less = start_non if start_non < start_dep else start_dep
            start_greater = start_dep if start_non < start_dep else start_non
            
            stop_less = stop_non if start_non < start_dep else stop_dep
            stop_greater = stop_dep if start_non < start_dep else stop_non
            
            token_less = token_non if start_non < start_dep else token_dep
            token_greater = token_dep if start_non < start_dep else token_non
            
            text_symbols[start_less:stop_less] = token_greater
            diff = len(token_greater) - (stop_less - start_less)
            text_symbols[start_greater + diff:stop_greater + diff] = token_less
            order_spoiled_text.append(''.join(text_symbols))
        else:
            # если не нашли
            order_spoiled_text.append(text)
    
    return order_spoiled_text

In [ ]:
# генерация состязательных примеров
adversial_examples_order = extracted_test
adversial_examples_order['order_spoiled_text'] = order_bkd_extract(adversial_examples_order, 'text')

# оставляем только те, которые были изменены
mask = adversial_examples_order['order_spoiled_text'] != adversial_examples_order['text']
only_spoiled_text = adversial_examples_order[mask]

In [ ]:
# показатели use_metric
_, use_result_order_bert = use_score(
    only_spoiled_text['text'],
    only_spoiled_text['order_spoiled_text'],
    use_bert_encoder=True,
    model=bert
)
_, use_result_order = use_score(
    only_spoiled_text['text'],
    only_spoiled_text['order_spoiled_text']
)

use_result_order_bert, use_result_order

In [ ]:
# вычисляем accuracy на испорченном датасете
sentidata = SentimentData(
                dataframe=adversial_examples_order,
                tokenizer=rubert_tokenizer,
                max_len_sent=max_len_sent_test * 2,
                mode='test',
                col_name='order_spoiled_text'
            )
adversial_score_order = calculate_accuracy(bert, sentidata)

# вычисляем accuracy на исходном датасете
sentidata = SentimentData(
                dataframe=adversial_examples_order,
                tokenizer=rubert_tokenizer,
                max_len_sent=max_len_sent_test * 2,
                mode='test',
                col_name='text'
            )
adversial_score_test = calculate_accuracy(bert, sentidata)